# change to root dir

In [1]:
import os

In [2]:
directory_path = 'C:/WINE_QUALITY_ML_PROJ'

In [3]:
os.chdir(directory_path)

In [4]:
%pwd

'C:\\WINE_QUALITY_ML_PROJ'

# create entity (class skeleton)

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

# create source configuration

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,
            schema_file_path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath) # get the whole configuration
        self.params = read_yaml(params_file_path) # get the schema
        self.schema = read_yaml(schema_file_path) # get the parameters

        create_directories([self.config.artifacts_root]) # create artifacts folder, if exist already then don't throw error, handle this in utils.common.create_directories code


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir]) # create model_trainer dir under artifacts bcoz holding the path -> artifacts/model_trainer

        model_trainer_config = ModelTrainerConfig( # creating object of class
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config # returning the object

# create the components

In [9]:
import pandas as pd
import os
import joblib
from sklearn.linear_model import ElasticNet
from mlProject.logging.logging import logger

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)


        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

        

# create the pipeline

In [11]:
try:
    config = ConfigurationManager() # creating an object so calling the configuration manager
    # here don't be confused like in the ConfigurationManager Class Constructor we defined 3 arguments but here we are not passing any, So why it is not showing error?
    # Because we assign all 3 arguments with "default" values i.e. path of config, params and schema yaml files
    model_trainer_config = config.get_model_trainer_config() # fetching the object from config manager
    model_trainer_config = ModelTrainer(config = model_trainer_config) # creating the component related to object, only make 1 component
    model_trainer_config.train() # fetching that component
except Exception as e:
    raise e

[2024-03-09 16:28:53,842: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-09 16:28:53,843: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-09 16:28:53,851: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-09 16:28:53,859: INFO: common: created directory at: artifacts]
[2024-03-09 16:28:53,859: INFO: common: created directory at: artifacts/model_trainer]
